In [ ]:
# %pip install requests

In [1]:
import json
import ast
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
DIR_TRAIN  = './input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = './input/sf-dst-car-price-prediction/'
test = pd.read_csv(DIR_TEST+'test.csv')

In [2]:
test['vehicleConfiguration'].iloc[:10]


0                LIFTBACK ROBOT 1.2
1           LIFTBACK MECHANICAL 1.6
2                LIFTBACK ROBOT 1.8
3            LIFTBACK AUTOMATIC 1.6
4            LIFTBACK AUTOMATIC 1.8
5                LIFTBACK ROBOT 2.0
6         ALLROAD_5_DOORS ROBOT 1.8
7           LIFTBACK MECHANICAL 1.6
8    ALLROAD_5_DOORS MECHANICAL 1.4
9    ALLROAD_5_DOORS MECHANICAL 1.8
Name: vehicleConfiguration, dtype: object

In [3]:
# искать в ключах json в т.ч. и имена столбцов, предварительно выбросив некоторые ненужные (image,description)
keystolook=['bodyType', 'brand', 'color', 
        'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType',  'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня']
valuetolook=[]
keyvaluepairtolook=[]
for col in keystolook:
    # print(test[col].value_counts()[:2])
    ll=test[test.brand=='BMW'][col].value_counts().index[:2].values.tolist()
    if str(ll[0]).__contains__('{'):keyvaluepairtolook.extend(ll) #словари в отдельный список
    else : valuetolook.extend(ll) #значения в другой список

# обработать немного по-другому разветвленный словарь - и его тоже потом разбить на ключи и значения
ll=test[test.brand=='BMW']['complectation_dict'].value_counts().index[:1].values[0]
keyvaluepairtolook.append(ll)

kvpairstofind=[] #преобразование строк в словари
for i in range(len(keyvaluepairtolook)):
    try:
        kvpairstofind.append(ast.literal_eval(keyvaluepairtolook[i]))
    except Exception as ex:
        print(keyvaluepairtolook[i],i," error:",ex)
# print(kvpairstofind)

tmpkey,tmpval=set(),set()
#разбор словарей отдельно на ключи и значения
def parsedictolists(dic):
    for k,v in dic.items():
        tmpkey.add(k)
        if type(v)==dict:
            parsedictolists(v)
        elif type(v)==list:
            for li in v:
                if type(li)==dict:
                    parsedictolists(li)
                else: tmpval.add(li)
        else: tmpval.add(v)

for dic in kvpairstofind:
    parsedictolists(dic)
#формирование списка ключей и списка значений
keystolook.append('available_options') #этот ключ визуально замечен, тоже добавить для поиска цепочки
tmpkey.update(set(keystolook))
tmpval.update(set(valuetolook))

# здесь собраны неповторяющиеся ключи и значения, которые надо поискать в jsone
print(tmpkey)                
print(tmpval)                

#две ошибки с распарсиванием значения true пока не рассматривать


{"leather":true} 0  error: malformed node or string: <_ast.Name object at 0x000000000A90F0D0>
{"fabric-seats":true} 1  error: malformed node or string: <_ast.Name object at 0x000000000A90F340>
{'transmission', 'Состояние', 'acceleration', 'power', 'enginePower', 'name', 'vehicleTransmission', 'power_kvt', 'model_info', 'sell_id', 'fuel_rate', 'fuelType', 'mileage', 'color', 'brand', 'displacement', 'nameplate', 'productionDate', 'morphology', 'equipment_dict', 'semantic_url', 'available_options', 'priceCurrency', 'clearance_min', 'Руль', 'id', 'code', 'modelDate', 'ПТС', 'Привод', 'ru_name', 'gear_type', 'human_name', 'engineDisplacement', 'parsing_unixtime', 'super_gen', 'Владение', 'vehicleConfiguration', 'bodyType', 'engine_type', 'numberOfDoors', 'model_name', 'Таможня', 'Владельцы', 'vendor'}
{4, 5, 6.8, 6.9, 6.2, 10.1, '9265070', 'AUTOMATIC', 'esp', 'abs', '5ER', 'Дубликат', 'механическая', '8494869', 'climate-control-1', 'lock', 'light-cleaner', '3 серии', 'wheel-configuration2'

In [4]:
from collections import defaultdict
import requests 
#данные, просмотренные в мозила- какой заголовок посылать для получения страницы авто.ру
false=False
url='https://auto.ru/-/ajax/desktop/listing/'
js={"category":"cars","section":"all","has_image":false,"customs_state_group":"DOESNT_MATTER","year_to":2020,"page":1,"catalog_filter":[{"mark":"BMW"}],"geo_radius":100,"geo_id":[213]}
brheader='''
Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
x-client-app-version: 8bd42497b89
x-client-date: 1642233376701
x-csrf-token: 9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5
x-requested-with: XMLHttpRequest
x-page-request-id: 18045d227a4fdec2917de7935ca9602e
x-yafp: {"a1":"+9mUFOhH1rWWmw==;0","a2":";1","a3":"RJnfx2MboJswCanp/pTWfA==;2","a4":"StYko3zFWsNlnqVqSmk8wtiCNceQDVXno+Ty8egp/wNyPQ==;3","a5":"wjYu92t/8PFxSg==;4","a6":"Vf8=;5","a7":"KvAfsVXDGAb9HQ==;6","a8":"eShrxqXZZi8JBw==;7","a9":"JW6wMh5dW7W7xg==;8","b1":"WMS1X811zcyfTA==;9","b2":"yyjQa/YhmCMLeQ==;10","b3":"naIppb8qXU+Xlw==;11","b4":"+X3co2U+4Iw=;12","b5":"ogGn4VtDnhQ22Q==;13","b6":"+z2Q4WofX4A=;14","b7":"EoTKve8VDMvjHA==;15","b8":"yUok16zaJal5sA==;16","b9":"siovMEfX8g2Gew==;17","c1":"1mBBIQ==;18","c2":"EdUNHgSksixRjnYOmi68eyuv;19","c3":"db46n2HIzPM1qzgUsfYyHffB;20","c4":"EGxT9UTLOBc=;21","c5":"idSfAP4K5Uo=;22","c6":"FJTeOQ==;23","c7":"tRonCBgZhOs=;24","c8":"M7FsDWGzADg=;25","c9":"pIwxReY43sw=;26","d1":"jPxdojxZZZeSF0iatEsue0X2DnlVMp8dIvaMllbb8LyIXVp/iZrcCU36Bg82evui8HjdaSl/;27","d2":"VDg=;28","d3":"qu1pusEc2NS/fg==;29","d4":"ykBYQZNXFv4=;30","d5":"yxfY97fzUW+BnQ==;31","d7":"RNgWvmQheUM=;32","d8":"JjI+RtXpluifFjkei3ajCpu/QyctL6CvHBo=;33","d9":"r1lNqkxAKho=;34","e1":"d3hBxv/sTectjA==;35","e2":"io2TNyuAibhRog==;36","e3":"dcX6jLdzgCIYhg==;37","e4":"zTY/oGADkWI=;38","e5":"vrzfRKzIg2ZSBA==;39","e6":"6iYhrYEdONs=;40","e7":"U+LLxN5sKjo=;41","e8":"jlUCPDEwNGA=;42","e9":"Insn4K4Iqc4=;43","f1":"r56Ndh8a2QXDkQ==;44","f2":"Qpll3AumPiI=;45","f3":"P5JubqtBinrKGQ==;46","f4":"GB3B60t7OF4=;47","f5":"Mq95O0hQKB2SGQ==;48","f6":"OK73ejIRtckzmA==;49","f7":"C7Jo/l+7Ggjvpw==;50","f8":"BH7thbLCvcJWtQ==;51","f9":"8UPV/s0K/Ak=;52","g1":"jTDuAwUJxtE=;53","g2":"n3Pb4Rhk4qQp5w==;54","g3":"sHw8QnFKhdE=;55","g4":"hanEck+CRe8=;56","g5":"VAJXAcn/KCE=;57","g6":"pMypKwIOiJhyTQ==;58","g7":"CwTU8X2eT3E=;59","g8":"4d26J5yVR4U=;60","g9":"wK7945Zt8i8=;61","h1":"9AInkX3tt0KXng==;62","h2":"l3Rdcw4wZUwafQ==;63","h3":"SR6HwaeL++QM2Q==;64","h4":"l+dC+vblTJZnEg==;65","h5":"UV8OIr2z61s=;66","h6":"X1oSmLCKI52Cfw==;67","h7":"wOjd9QM7NZNcQW/0JjWRD5T52Y3bodsWXoNp2YcH8Wyukhb1UTew/B0cvezKs9QEEap5mU2XIvCJg6uxPf9AsYXo0PUDOyKTCEEt9DM1ig+Q+eGNmqHLFlSDYtmXB6hs55JZ9UY38fwYHLzs37PcBFSqe5lGl2zwnIPlsXT/QbGT6N/1FDsnkx1BbvQiNcIPsPngjd6hxhZUg0TZiwfmbP2SU/VKN6X8Xxw=;68","h8":"/BCuzEtOychW7w==;69","h9":"FBEsYWHV4nvLaw==;70","i1":"AYXrFKDs94o=;71","i2":"Z2r3MleTRcZt9g==;72","i3":"9jhq+iOUK7PxBg==;73","i4":"9HWV3QWKr7TJlg==;74","i5":"UhPNwfrW5YtxyA==;75","z1":"X/S2w8juiivuh5tNnyXFf7ZicLUgsf6fa1fi2Mu2ToIuEE4sm+x+7345sXiWevDxY2Gp6h6fOm/8vWzuIJsEDQ==;76","z2":"89XLvCGRrsGF3dtSHbko1z7GgAO/o/OEW5WlM5ZnexjHx3hvf4rcgDMhYhLUl2gpLH82unkBpFL4IbeSMiQayw==;77","z3":"m9LfDh+/njvJ9w==;78","z6":"+NMrQoLE6YGRZuuA;79","z7":"vpnOcULUNQV8F1xf;80","z8":"Q+PA64DATB+AZg==;81","z9":"dEoDzd8viQ3P3g==;82","y1":"zkSOPfJmbbejBw==;83","y2":"GoY3Mls8TN56Ow==;84","y3":"WnalUUEBHD55VA==;85","y4":"ujyAI5y9XDRYpQ==;86","y5":"8KeCDkNpsaaNEg==;87","y6":"X/StZ62O0x6adQ==;88","y7":"BaDX/sSjTRmNPlJB;89","y8":"5xtR70OjoKUzZQ==;90","y9":"dogOajbNqQSnJg==;91","y10":"urOhyDWLxgQ20g==;92","x1":"5B3uMq7G7GmeIA==;93","x2":"8E18p8rSLsJt+A==;94","x3":"qyIHLcmgjts7Yw==;95","x4":"UqPDrKNBiZA+4A==;96","x5":"q8f9mu/g0BJ1KFS0;97","z5":"MLFA/oKKRWo=;98","z4":"poxmqqYtBdfiNOjClbQ=;99","v":"6.3.1","pgrdt":"tIPfIJoihG27AU6WOtGWU4Ft144=;100","pgrd":"5L+ttwRwCk2f75zf0ZSEywqTgH3k42gYgEvTyb/RL2pTcvJugUYEm/lGL8I60EEVEc9dfWU/bjpewbufCBOR1UgueETv/Kmglh9FrUxQFZwSqgsrZwvtV7LAp/t2eXwVf5dpqPjXBI5fretcUlR+Ah8T5itlg7QXQ7b5UuFTS5/cR43nDhNSvBc/3KMlE5082xCk2gOHclPDJOYEPKyBNznzYqM="}
content-type: application/json
Origin: https://auto.ru
Content-Length: 181
Connection: keep-alive
Cookie: autoru_sid=a%3Ag61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1%7C1642075015356.604800.NyNBXPK57aO3A4-Fo8IjLg.mMqAMysiCxrZlvcJeohRogrbR5jwiGpAsotk9SwB-PI; autoruuid=g61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1; suid=a06c2b573007272e470c6bb6f7511ff2.e5976b34552661d0ab47d79e4fc53c7e; _yasc=JYqgy/CgqEsxvyIjwOjq/i7aIMB4KhoRNSGMgygRRcrkVt4/ELo=; yuidlt=1; yandexuid=3644335621586616841; yandex_login=; mda2_beacon=1642093553695; crookie=ZZh3+UR44UJlHEcXIHCHEhLtuL1SWoMmxMgXvbZOjEKfcgYC+Qfo8QOA7oa+xrYCTmpAkGV1DUCRG8wwHzaMSsouZ9M=; cmtchd=MTY0MjA3NTAyNDQ0Nw==; gids=213; gradius=100; los=1; bltsr=1; _csrf_token=9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5; from_lifetime=1642233370088; from=direct; ys=c_chck.1432299411; autoru-visits-count=1; X-Vertis-DC=vla
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: same-origin
Sec-Fetch-Site: same-origin
'''
brheader=brheader.split('\n')
brheaderdict={}
for item in brheader:
	if not(item):continue
	kv=item.split(': ')
	key,value=kv[0],kv[1]
	brheaderdict[key]=value
pass

def responsedic(page,js)->dict:
    js['page']=page
    response = requests.post(url,json=js,headers=brheaderdict)  
    if response.status_code==200:
        return defaultdict(lambda: "нет в jsone",response.json())

# datadic=responsedic(1,js)
# with open("temp.json","w") as f:
#     json.dump(datadic,f)


In [199]:
#проба - поправить куски текста с авто.ру, которые имеют нарушенную структуру
# import json
# import re

# class LazyDecoder(json.JSONDecoder):
#     def decode(self, s, **kwargs):
#         regex_replacements = [
#             (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
#             (re.compile(r',(\s*])'), r'\1'),
#         ]
#         for regex, replacement in regex_replacements:
#             s = regex.sub(replacement, s)
#         return super().decode(s, **kwargs)


In [ ]:
#в возвращаемых данных json есть некорректные данные
# поправить их, чтобы просмотрщиками json можно было смотреть
import copy

datadic=responsedic(1,js)

datadic_=copy.deepcopy(datadic)

#используя итератор для зеркального словаря datadic, удаление из datadic_ больших кусков текста, 
#в них попадаются несбалансированные кавычки, бэкслэши и хрен знает что еще и json их не открывает
def parsedictremovechunk(mirrordic,dic,isfirst=True):
    for k,v in mirrordic.items():
        if type(v)==dict:
            parsedictremovechunk(v,dic[k],isfirst)
        elif type(v)==list:
            curind=0
            for li in v:
                if type(li)==dict:
                    parsedictremovechunk(li,dic[k][curind],isfirst)
                    curind+=1
        elif type(v)==str and len(v)>150:
            if isfirst:
                if dic[k]==v: dic[k]="text removed"
                else: print("не зеркало")
            else: print("проверка {},{}\n{}".format(k,dic[k],v))
        else: 
            pass
            # print("неожиданный тип",type(v))

parsedictremovechunk(datadic,datadic_)
#проверка
parsedictremovechunk(datadic,datadic_,False)



In [ ]:
# попытки прочитать в json словарь из подготовленной строки
tmp=datadic_.__str__()
# tmp=tmp.replace("'",'"')
# tmp=tmp.replace("False",'false')
# tmp=tmp.replace("True",'true')
# tmp=tmp.replace("None",'null')
# js=json.loads(tmp)
# js=json.loads(tmp, cls=LazyDecoder)
# datadic_=ast.literal_eval(tmp)
tmp=json.dumps(datadic_)
tmp


In [215]:
# задача - найти нужные данные в json, возвращаемом с портала авто.ру, чтобы понимать, откуда их понадергать для нового датасета
# анализ проводить по данным для БМВ - это не сбор данных, а только поиск, откуда вытягивать
# проанализировать возвращаемый посредством post кусок данных json на включение требуемых признаков из тестового датасета test
# datadic -словарь ответ json портала авто.ру 
# debugpath цепочка запроса к datadic, где нашлось что-то похожее
# rootkey - для пропуска всех корневых ключей и просмотра только списка офферов
def deepintodict(datadic,debugpath,rootkey="")->str:
    def comparek(keystofind,key,value,debugpath)->str:
        for keytofind in keystofind:
            if keytofind==key:
                existkey.add(keytofind)
                if not deepkeys.__contains__(debugpath):
                    deepkeys.append(debugpath)
                    print(debugpath+"='{}'".format(value))
                    return "['{}']='{}'".format(key,value)
        return ''
    def comparev(valuestofind,key,value,debugpath)->str:
        for valuetofind in valuestofind:
            if valuetofind==value:
                existval.add(valuetofind)
                if not deepkeys.__contains__(debugpath):
                    deepkeys.append(debugpath)
                    print(debugpath+"='{}'".format(value))
                    return "['{}']='{}'".format(key,value)
        return ''

    res1,res="",""
    try:
        for key,value in datadic.items():
            if rootkey and rootkey!=key:continue #рассмотреть только один offers - из корневых ключей
            td=type(value)
            debugpath_=debugpath+"['{}']".format(key)
            if td==dict:
                res1=comparek(keystofind,key,value,debugpath_)
                res=deepintodict(value,debugpath_)
                if res: 
                    res1="['{}']{}".format(key,res)
                    pass
                continue
            if td==list:
                if len(value)>0 and type(value[0])==dict: #если список словарей, то пройти по словарям
                    count=0
                    for item in value:
                        res=deepintodict(item,"{}[{}]".format(debugpath_,'x'))
                        if res: 
                            res1="['{}'][{}]{}".format(key,count,res)
                            pass
                        count+=1
                        continue
                else: #если список значений один ключ key
                    res1=comparek(keystofind,key,value,debugpath_)
                    for valinlist in value:
                        res1=comparev(valuestofind,key,valinlist,debugpath_)
            else:
                res1=comparek(keystofind,key,value,debugpath_)
                res1=comparev(valuestofind,key,value,debugpath_)
    except Exception as ex:
        print(res1,res,": здесь ошибка {0} ".format(ex))
    return res1

#сюда сохранять ключи и значения, которые нашлись в jsone
existkey=set()
existval=set()
deepkeys=[] #рабочий список для хранения цепочки ключей без учета индексов (индексы заменяются на х) - чтобы просто найти хоть одну цепочку ключей и понять, нужно это или нет
#взять нужные столбцы из test и поискать совпадения, а так же добавить пару ключей для больших списков разных опций, которые визуально выявлены при просмотре датасета test
keystofind=list(tmpkey) 
# взять наиболее часто встречающиеся значения из датасета test 
valuestofind=list(tmpval)
import time
t0=0
for page in range(1,10):
    #получить с сервера страницу page в словарь
    while time.time_ns() - t0<500000000: #проверять время, чтобы не получить отказ от сервера
        time.sleep(0.1)
    datadic=responsedic(page,js)
    t0 = time.time_ns()  # начальное время старта
    #найти в словаре ключи и значения и соответствующие им полные цепочки ключей, где поискать нужные данные в этом jsone
    res=deepintodict(datadic,"",rootkey='offers')
    #теперь пополнились множества existkey,existval
    #определить, что нашлось и что- нет
    difk=set(keystofind).difference(existkey)
    difv=set(valuestofind).difference(existval)
    print("\nне найдены ключи {}из{}".format(len(difk),len(keystofind)),difk)
    print("не найдены значения {}из{}".format(len(difv),len(valuestofind)),difv)
    print("\n\n")

    #вызвать следующую страницу с авто.ру и поискать оставшиеся ключи из начения там
    #на следующей странице искать не все, а среди оставшихся
    keystofind=list(difk) 
    valuestofind=list(difv)


['offers'][x]['owner_expenses']['transport_tax']['horse_power']='249'
['offers'][x]['additional_info']['days_in_stock']='4'
['offers'][x]['additional_info']['days_on_sale']='4'
['offers'][x]['id']='1106507931'
['offers'][x]['seller']['location']['region_info']['id']='213'
['offers'][x]['seller']['location']['region_info']['name']='Москва'
['offers'][x]['seller']['location']['timezone_info']['name']='Europe/Moscow'
['offers'][x]['seller']['name']='id13039842'
['offers'][x]['state']['mileage']='51000'
['offers'][x]['state']['external_panorama']['published']['id']='1419387375-1641654220784-sDduG'
['offers'][x]['state']['image_urls'][x]['name']='1'
['offers'][x]['vehicle_info']['configuration']['id']='20158770'
['offers'][x]['vehicle_info']['configuration']['doors_count']='5'
['offers'][x]['vehicle_info']['configuration']['human_name']='Внедорожник 5 дв.'
['offers'][x]['vehicle_info']['complectation']['id']='21075655'
['offers'][x]['vehicle_info']['complectation']['name']='xDrive30d M Spor

In [227]:
# далее изучаем json и собираем для столбцов ключи:

        #bodyType
        # ['offers'][x]['vehicle_info']['configuration']['human_name']='Универсал 5 дв.'
        # перевести в нижний регистр
        # brand
        # ['offers'][x]['vehicle_info']['mark_info']['code']='BMW'
        # ['offers'][x]['vehicle_info']['mark_info']['name']='BMW'
        # color ???? переводить как-то
        #['color_hex']
        # ['offers'][x]['vehicle_info']['complectation']['vendor_colors'][x]['stock_color']['name_ru']='белый'
        # complectation_dict
        # ['offers'][x]['vehicle_info']['complectation']
        # ['offers'][x]['vehicle_info']['complectation']['available_options']='['airbag-passenger', 'lock', 'electro-mirrors', 'mirrors-heat', 'seat-transformation', 'wheel-power', 'fabric-seats', 'abs', 'wheel-leather', 'tinted-glass', 'audiopreparation', 'ashtray-and-cigarette-lighter', 'front-centre-armrest', 'body-mouldings', '15-inch-wheels', 'airbag-driver', 'isofix', 'electro-window-front', 'paint-metallic', 'ptf', 'migration-flag', 'power-child-locks-rear-doors', 'wheel-configuration2', 'wheel-configuration1', 'immo']'
       
        # engineDisplacement (2.0 LTR)
        # #только литраж найден в компоновочной записи (как 2.0d соответствует литражу)
        # ['offers'][x]['vehicle_info']['lk_summary']='"2.0d AT (150 л.с.), внедорожник, полный, дизель"'

        # enginePower (249 N12- надо еще N12 пристегнуть для однотипности с датасетом test)
        # ['offers'][x]['owner_expenses']['transport_tax']['horse_power']='249'
        # ['offers'][x]['vehicle_info']['tech_param']['power']='249'
        # ['offers'][x]['vehicle_info']['tech_param']['power_kvt']='183'

        # equipment_dict 
        # # найдено визуально в json
        # ['offers'][x]['vehicle_info']['equipment']
        # fuelType (в датасете по русски)
        # в jsone по-русски толко в lk_summary 
        # ['offers'][x]['vehicle_info']['tech_param']['engine_type']='DIESEL'

        # mileage
        # ['offers'][x]['state']['mileage']='51000'

        # productionDate
        # ['offers'][x]['documents']['year']='2009'

        # modelDate не нашел, что это за дата
        # ['offers'][x]['vehicle_info']['super_gen']['year_from']='2013'
        # ['offers'][x]['vehicle_info']['super_gen']['year_to']='2012'

        # model_info
        # ['offers'][x]['vehicle_info']['model_info']='{'code': 'X6', 'name': 'X6', 'ru_name': 'Х6', 'morphology': {}, 'nameplate': {'code': '9265282', 'name': '30d', 'semantic_url': '30d'}}'

        # model_name
        # ['offers'][x]['vehicle_info']['model_info']['name']

        # name 1.6 AT (110 л.с.) надо как-то клеить
        # ['offers'][x]['vehicle_info']['tech_param']['human_name']='30d 3.0d AT (249 л.с.) 4WD'

        # numberOfDoors
        # ['offers'][x]['vehicle_info']['configuration']['doors_count']='5'

        # parsing_unixtime
        # в jsone не нашел ничего похожего


        # super_gen словарь
        # ['offers'][x]['vehicle_info']['tech_param']

        # vehicleConfiguration
        # похоже на склейку трех параметров
        # ['offers'][x]['vehicle_info']['configuration']['body_type']
        # ['offers'][x]['vehicle_info']['tech_param']['transmission']='AUTOMATIC'
        # и литраж надо прицепить сюда же

        # vehicleTransmission русских атрибутов в jsone не нашлось, надо сопоставлять 
        # ['offers'][x]['vehicle_info']['tech_param']['transmission']='AUTOMATIC' автоматическая

        # vendor
        # ['offers'][x]['vehicle_info']['vendor']='EUROPEAN'

        # Владельцы 1 владелец,2 владельца,3 или более
        # ['offers'][x]['documents']['owners_number']='4' надо допилить

        # Владение
        # ['offers'][x]['owner_expences']['holding_period_month']=12 везде одинаково вроде
        # значит вычислять как то

        # ПТС СОПОСТАВЛЯТЬ    оРИГИНАЛ дУБЛИКАТ
        # ['offers'][x]['documents']['pts_original']=true
        # ['offers'][x]['documents']['pts']=ORIGINAL

        # Привод Передний,Полный,Задний сопоставлять
        # ['offers'][x]['vehicle_info']['tech_param']['gear_type']='ALL_WHEEL_DRIVE'
        # ['offers'][x]['vehicle_info']['tech_param']['gear_type']='REAR_DRIVE'

        # Руль Левый Правый -сопоставлять
        # ['offers'][x]['vehicle_info']['steering_wheel']=LEFT

In [12]:
# %pip install -U ipykernel

  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.0.5
    Uninstalling traitlets-5.0.5:
      Successfully uninstalled traitlets-5.0.5
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: ipython
    Found existing installation: ipython 7.19.0
    Uninstalling ipython-7.19.0:
      Successfully uninstalled ipython-7.19.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.3.4
    Uninstalling ipykernel-5.3.4:
      Successfully uninstalled ipykernel-5.3.4



ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [7]:
import time
from collections import defaultdict
#заполнить датасет из словаря, полученного с сервера
def feelDF(dic,df)-> pd.DataFrame:
    tst=test[test.brand=='BMW']
    # словари для сопоставления в основном английских и русских наименований, когда в ответе jsone не нашлось ничего подходящего (а в html почему-то есть)
    Wheel=defaultdict(lambda: "absent",{'LEFT':'Левый','RIGHT':'Правый'})
    Privod=defaultdict(lambda: "absent",{'ALL_WHEEL_DRIVE':'Полный','REAR_DRIVE':'Задний'})
    Pts=defaultdict(lambda: "absent",{'ORIGINAL':'оригинал'})
    Owner=defaultdict(lambda: "absent",{1:'1 владелец',2:'2 владельца',3:'3 или более'})
    color=defaultdict(lambda: "absent",{'040001':'Чёрный','FAFBFB':'Белый'})
    for offerInd in range(len(dic['offers'])):
        dto=dic['offers'][offerInd]
        dto_vehicle_info=defaultdict(lambda: "key absent!",dto['vehicle_info'])
        print(type(dto))
        new_row = {'bodyType':dto['vehicle_info']['configuration']['human_name'].lower(),
                    'bodyType_':tst['bodyType'].iloc[offerInd],
                    'brand':dto['vehicle_info']['mark_info']['name'],
                    'brand_':tst['brand'].iloc[offerInd],
                    'color':color[dto['color_hex']],
                    'color_':tst['color'].iloc[offerInd],
                    'complectation_dict':dto['vehicle_info']['complectation'].__str__(),
                    'complectation_dict_':tst['complectation_dict'].iloc[offerInd],
                    
                    'engineDisplacement':dto_vehicle_info['lk_summary'][:3]+" LTR",
                    'engineDisplacement_':tst['complectation_dict'].iloc[offerInd],
                    
                    'enginePower':dto['vehicle_info']['tech_param']['power']+" N12",
                    'enginePower_':tst['enginePower'].iloc[offerInd],
                    
                    'equipment_dict':dto['vehicle_info']['equipment'].__str__(),
                    'equipment_dict_':tst['equipment_dict'].iloc[offerInd],
                    'fuelType':dto['vehicle_info']['tech_param']['engine_type'],
                    'fuelType_':tst['fuelType'].iloc[offerInd],
                    'mileage':dto['state']['mileage'],
                    'mileage_':tst['mileage'].iloc[offerInd],
                    
                    'productionDate':dto['documents']['year'],
                    'productionDate_':tst['productionDate'].iloc[offerInd],
                    'modelDate':dto['vehicle_info']['super_gen']['year_from'],
                    'modelDate_':tst['modelDate'].iloc[offerInd],
                    'model_info':dto['vehicle_info']['model_info'].__str__(),
                    'model_info_':tst['model_info'].iloc[offerInd],

                    'model_name':dto['vehicle_info']['model_info']['name'],
                    'model_name_':tst['model_name'].iloc[offerInd],
                    'name':dto['vehicle_info']['tech_param']['human_name'],
                    'name_':tst['name'].iloc[offerInd],
                    'numberOfDoors':dto['vehicle_info']['configuration']['doors_count'],
                    'numberOfDoors_':tst['numberOfDoors'].iloc[offerInd],
                    'parsing_unixtime':dto['additional_info']['creation_date'],
                    'parsing_unixtime_':tst['parsing_unixtime'].iloc[offerInd],

                    'super_gen':dto['offers'][x]['vehicle_info']['tech_param'].__str__(),
                    'super_gen_':tst['super_gen'].iloc[offerInd],
                    'vehicleConfiguration':'{}{}{}'.format(dto['vehicle_info']['configuration']['body_type'],dto['vehicle_info']['tech_param']['transmission'],dto['vehicle_info']['lk_summary'][:3]),
                    'vehicleConfiguration_':tst['vehicleConfiguration'].iloc[offerInd],
                    'vehicleTransmission':dto['vehicle_info']['tech_param']['transmission'],
                    'vehicleTransmission_':tst['vehicleTransmission'].iloc[offerInd],
                   
                    'vendor':dto['vehicle_info']['vendor'],
                    'vendor_':tst['vendor'].iloc[offerInd],
                    'Владельцы':Owner[dto['documents']['owners_number']],
                    'Владельцы_':tst['Владельцы'].iloc[offerInd],
                    'Владение':'надо вычислять',
                    'Владение_':tst['Владение'].iloc[offerInd],
                    'ПТС':Pts[dto['documents']['pts']],
                    'ПТС_':tst['ПТС'].iloc[offerInd],
                    'Привод':Privod[dto['vehicle_info']['tech_param']['gear_type']], 
                    'Привод_':tst['Привод'].iloc[offerInd],
                    'Руль':Wheel[dto['vehicle_info']['steering_wheel']],
                    'Руль_':tst['Руль'].iloc[offerInd],
                    }
        df=df.append(new_row,ignore_index=True)
    return df

df=pd.DataFrame()
t0=0
datadic={}
for page in range(1,2):
    #получить с сервера страницу page в словарь
    while time.time_ns() - t0<500000000: #проверять время, чтобы не получить отказ от сервера
        time.sleep(0.1)
    datadic=responsedic(page,js)
    t0 = time.time_ns()  # начальное время старта
    df=feelDF(datadic,df)
    pass



<class 'dict'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [6]:
tmp=json.dumps(datadic)
tmp

'{"status": "SUCCESS", "search_parameters": {"has_image": false, "year_to": 2020, "catalog_filter": [{"mark": "BMW"}], "customs_state_group": "DOESNT_MATTER", "section": "all", "category": "cars"}, "hiddenOffers": [], "offers": [{"availability": "IN_STOCK", "badges": ["\\u041e\\u0434\\u0438\\u043d \\u0432\\u043b\\u0430\\u0434\\u0435\\u043b\\u0435\\u0446", "\\u041c\\u0430\\u043a\\u0441\\u0438\\u043c\\u0430\\u043b\\u044c\\u043d\\u0430\\u044f \\u043a\\u043e\\u043c\\u043f\\u043b\\u0435\\u043a\\u0442\\u0430\\u0446\\u0438\\u044f", "\\u041e\\u0431\\u0441\\u043b\\u0443\\u0436\\u0438\\u0432\\u0430\\u043b\\u0430\\u0441\\u044c \\u0443 \\u0434\\u0438\\u043b\\u0435\\u0440\\u0430"], "category": "cars", "color_hex": "0000CC", "delivery_info": {}, "description": "\\u0414\\u0418\\u041b\\u0415\\u0420\\u0410\\u041c, \\u041f\\u0415\\u0420\\u0415\\u041a\\u0423\\u041f\\u0429\\u0418\\u041a\\u0410\\u041c \\u0418 \\u0421\\u0410\\u041b\\u041e\\u041d\\u0410\\u041c \\u041d\\u0415 \\u0417\\u0412\\u041e\\u041d\\u04

In [ ]:
df.sample(38)

In [ ]:


# проанализировать возвращаемый посредством post кусок данных json на включение требуемых признаков из тестового датасета test
deepkeys=[]
# datadic -словарь ответ json портала авто.ру 
# keytofind поиск одноименного ключа
# debugpath цепочка запроса к datadic, где нашлось что-то похожее
# rootkey - для пропуска всех корневых ключей и просмотра только списка офферов
def deepintodict(datadic,keytofind,debugpath,rootkey="")->str:
    res1=""
    try:
        for key,value in datadic.items():
            if rootkey and rootkey!=key:continue #рассмотреть только один offers - из корневых ключей
            td=type(value)
            # print('type:{0} {1}'.format(type(value),td))
            
            if td==dict:
                res=deepintodict(value,keytofind,"{}['{}']".format(debugpath,key))
                if res: 
                    res1="['{}']{}".format(key,res)
                    pass
                # print(debugpath+'/'+key)
                continue
            if td==list:
                if len(value)>0 and type(value[0])==dict: #если список словарей, то пройти по словарям
                    count=0
                    for item in value:
                        debugpathnext="{}['{}'][{}]".format(debugpath,key,'x')
                        res=deepintodict(item,keytofind,debugpathnext)
                        if res: 
                            res1="['{}'][{}]{}".format(key,count,res)
                            pass
                        count+=1
                        continue
                else: #если список значений, то сразу проверить ключ
                    if keytofind==key:
                        if not deepkeys.__contains__(debugpath):
                            deepkeys.append(debugpath)
                            print(debugpath+"['{0}']='{1}'".format(key,value))
                        return "['{0}']='{1}'".format(key,value)
            else:
                if keytofind==key:
                    if not deepkeys.__contains__(debugpath):
                        deepkeys.append(debugpath)
                        print(debugpath+"['{0}']='{1}'".format(key,value))
                    return "['{0}']='{1}'".format(key,value)
    except Exception as ex:
        return ": здесь ошибка {0} ".format(ex)
    return res1

cols=testcolumns.copy()
for col in testcolumns:
    deepkey=deepintodict(datadic_,col,"",rootkey='offers')
    if deepkey:
        print("{}:  {}{}".format(col,"datadic",deepkey))
        cols.remove(col)
print("не найдены ключи:",cols)
# color:  datadic['offers'][3]['salon']['place']['metro'][0]['lines'][0]['color']='#ffa8af'

In [8]:
# %pip install beautifulsoup4  

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Импортируем наши библиотеки    
from bs4 import BeautifulSoup    
import requests    
    
# Получаем данные, как и ранее    
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)    
    
# Теперь создадим объект BeautifulSoup, указывая html парсер    
page = BeautifulSoup(response.text, 'html.parser')    
    
# Всё готово, чтобы получать данные из страницы    
# Для начала получим title, отображающийся на закладках браузера    
print(page.title)  
# => <title>Робота-ленивца научили лазать по паутине из тросов</title>  
    
# Мы получили тэг. Чтобы достать из него текст, вызовем атрибут text    
page.title.text    
# => 'Робота-ленивца научили лазать по паутине из тросов'  

<title>Робота-ленивца научили лазать по паутине из тросов</title>


'Робота-ленивца научили лазать по паутине из тросов'

In [10]:
def wiki_header(url):
    response = requests.get(url)    
    page = BeautifulSoup(response.text, 'html.parser')  
    qq=page.find('h1')
    return qq.text
    
wiki_header('https://en.wikipedia.org/wiki/Operating_system')

'Operating system'

In [11]:

url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'  
    
response = requests.get(url)  
page = BeautifulSoup(response.text, 'html.parser')  
page.find('a')  
# => <a id="top"></a>  


links = page.find_all('a')  
# Посмотрим, сколько всего мы получили  
print(len(links))
# => 935
    
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  
# => ['Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal', 'Pascal Script']


930
['OptimJ', 'Orc', 'ORCA/Modula-2', 'Oriel', 'Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4']


In [12]:

# Получаем все элементы с тегом 'div' и классом 'div-col'  
all_blocks = page.find_all('div', class_='div-col')  
    
# Выбираем первый по счету блок  
first_block = all_blocks[0]  
# Берём оттуда ссылки (ограничимся первыми десятью)
links = first_block.find_all('a')
print([link.text for link in links[:10]])
# => ['1C:Enterprise programming language']


['A.NET (A#/A sharp)', 'A-0 System', 'A+ (A plus)', 'ABAP', 'ABC', 'ABC ALGOL', 'ACC', 'Accent (Rational Synergy)', 'Ace DASL (Distributed Application Specification Language)', 'Action!']


In [13]:
def get_actors(url):
    response = requests.get(url)  
    page = BeautifulSoup(response.text, 'html.parser')  
    act = page.find('div', class_='styles_actors__2zt1j')  
    acts=act.find_all('a',class_='styles_link__1dkjp')
    qq=[]
    for act in acts:
        qq.append(act.text)
    return qq

# get_actors('https://www.kinopoisk.ru/film/42326/')
get_actors('https://www.kinopoisk.ru/film/326/')

['Тим Роббинс',
 'Морган Фриман',
 'Боб Гантон',
 'Уильям Сэдлер',
 'Клэнси Браун',
 'Гил Беллоуз',
 'Марк Ролстон',
 'Джеймс Уитмор',
 'Джеффри ДеМанн',
 'Ларри Бранденбург']

In [17]:
# %pip install lxml
# %pip install pandas
# %pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url = 'https://www.cbr.ru/key-indicators/'  
# Таблица с драгметаллами оказалась третьей по счёту  
qq=pd.read_html(url)
print(qq[1])


                 0           1           2
0           валюта  13.01.2022  14.01.2022
1  Доллар США  USD      745277      745686
2        Евро  EUR      846709      854556


In [2]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests  
    
url = 'https://www.banki.ru/banks/ratings/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  

all_blocks = soup.find_all('table', class_='standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small')
data = all_blocks[0]
df = pd.read_html(str(data))[0]  


In [3]:
qq='bfc62b39bfc62b39bfc62b3964bfbceed9bbfc6bfc62b39de1caa2ac943f4855e92bd90'



In [4]:
import requests  
  
url = 'https://api.vk.com/method/users.get'   
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': qq, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  

'{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}'

In [5]:

from pprint import pprint  
    
pprint(response.json())  

    
user = response.json()['response'][0]  
    
# Выведем дату рождения  
print(user['bdate'])  
# => 10.10.1984  
    
# Выведем имя  
print(user['first_name'])  
# => Павел  


{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}
10.10.1984
Павел


In [6]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  
# => 1,2,3  
  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': qq, 'lang': 'ru'}  
  
pprint(requests.get(url, params=params).json()) 

1,2,3
{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


In [7]:
ids = ",".join(map(str, range(1, 501)))  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': qq}  
userdics=requests.get(url, params=params).json()['response'] 
print(userdics[0])

{'id': 1, 'first_name': 'Pavel', 'last_name': 'Durov', 'can_access_closed': True, 'is_closed': False, 'sex': 2}


In [8]:
df=pd.DataFrame.from_dict(userdics)
df.head(3)

,id,first_name,last_name,can_access_closed,is_closed,sex,deactivated
0,1,Pavel,Durov,True,False,2,NaN
1,2,Alexandra,Vladimirova,False,True,1,NaN
2,3,DELETED,,NaN,NaN,0,deleted


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 500 non-null    int64 
 1   first_name         500 non-null    object
 2   last_name          500 non-null    object
 3   can_access_closed  364 non-null    object
 4   is_closed          364 non-null    object
 5   sex                500 non-null    int64 
 6   deactivated        136 non-null    object
dtypes: int64(2), object(5)
memory usage: 27.5+ KB


In [10]:
len(df.index)
df[df.sex==1].sample(5)

,id,first_name,last_name,can_access_closed,is_closed,sex,deactivated
259,260,Anastasia,Grafova,True,False,1,NaN
202,203,Zoya,Barabanova,True,False,1,NaN
145,146,Alexandra,Muromtseva,True,False,1,NaN
354,355,Elena,Batsmanova,True,False,1,NaN
417,418,Yulia,Ernie,True,False,1,NaN


In [11]:
print(len(df[df.sex==1].index)/len(df[df.sex!=0].index))

0.4821852731591449


In [12]:

import requests  
url = 'https://api.vk.com/method/groups.getMembers'  
params = {  
    'group_id': 'vk',  
    'v': 5.95,  
    'access_token': qq  
}  
response = requests.get(url, params = params)  
data = response.json()  
print(data)  
# => {'response': {'count': 11240724, 'items': [5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 205, 219... 
    
len(data['response']['items'])  
# => 1000


{'response': {'count': 12241022, 'items': [5, 6, 19, 34, 47, 54, 79, 177, 193, 198, 212, 219, 239, 243, 254, 345, 404, 406, 407, 467, 485, 510, 550, 619, 628, 640, 643, 690, 702, 720, 721, 724, 744, 804, 809, 831, 832, 834, 847, 900, 905, 907, 914, 943, 952, 958, 966, 976, 979, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1185, 1188, 1290, 1301, 1333, 1334, 1336, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1531, 1543, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1868, 1869, 1887, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2145, 2150, 2195, 2201, 2202, 2230, 2236, 2273, 2281, 2294, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2515, 2527, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2896, 2

1000

In [13]:
users_for_checking = data['response']['items'][:20]  
print(users_for_checking)  

[5, 6, 19, 34, 47, 54, 79, 177, 193, 198, 212, 219, 239, 243, 254, 345, 404, 406, 407, 467]


In [14]:
import time
cycle=0
count = 1000  
offset = 0  
user_ids = []
max_count = 100000  
while offset < max_count:  
    cycle+=1
    # print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    print(cycle,sep=' ',end='')     
    params = {  
        'group_id': 'vk',  
        'v': 5.131,  
        'count': count,  
        'offset': offset,  
        'access_token': qq  
    }     
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
    
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
    time.sleep(0.5)
        
print('\ndone ',user_ids[99999])  
    
# user_ids == users_for_checking  


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100
done  6052530


In [15]:
import requests  
from pprint import pprint  
  
url = 'https://api.vk.com/method/wall.get'  
params = {  
    'domain': 'vk',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': qq,  
    'v': 5.95  
}  
response = requests.get(url, params = params)  
response.json()


{'response': {'count': 464,
  'items': [{'id': 1268470,
    'from_id': -22822305,
    'owner_id': -22822305,
    'date': 1640980531,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': '',
    'copy_history': [{'id': 393,
      'owner_id': 92933,
      'from_id': 92933,
      'date': 1640980164,
      'post_type': 'post',
      'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире те

In [16]:
response.json()['response']['items']

[{'id': 1268470,
  'from_id': -22822305,
  'owner_id': -22822305,
  'date': 1640980531,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': '',
  'copy_history': [{'id': 393,
    'owner_id': 92933,
    'from_id': 92933,
    'date': 1640980164,
    'post_type': 'post',
    'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также 

In [17]:
response.json()['response']['items'][0]

{'id': 1268470,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1640980531,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 393,
   'owner_id': 92933,
   'from_id': 92933,
   'date': 1640980164,
   'post_type': 'post',
   'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также автоматически

In [18]:
stats = {}  
              
for record in response.json()['response']['items'][:]:  
    title = record['text'][:30]  
    if title:  
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]  
pprint(stats)  

{'А если взглянуть в целом, то к': [272, 849, 255, 1640094467],
 'ВКонтакте всегда есть что посм': [196, 873, 87, 1640017040],
 'Запускаем собственное шоу в пе': [71, 1277, 452, 1640940328],
 'Каким был ваш 2021-й ВКонтакте': [591, 2103, 9749, 1640174402],
 'Мы приготовили для вас целую г': [168, 3903, 390, 1640712137],
 'Объявляем итоги первого розыгр': [591, 1289, 327, 1640871560],
 'Одни называют всё, что происхо': [126, 1072, 284, 1639491162],
 'Приближаем праздники вместе — ': [7041, 1160, 676, 1639590427],
 'Эти каникулы будут особенными ': [128, 1150, 153, 1640171709]}


In [19]:
import requests  
from pprint import pprint  

def get_items(gr='vk', token=''):
  
    url = 'https://api.vk.com/method/wall.get'  
    params = {  
    'domain': 'vk',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
    }  
    response = requests.get(url, params = params)  
    dicts=response.json()['response']
    return dicts

ds=get_items(gr='vk', token=qq)
print(ds['count'])
print(ds['items'][0])
def get_smm_index(gr='vk', token=''):
    sum=0              
    for record in ds['items'][:10]:  
        sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    return sum/ds['count']


464
{'id': 1268470, 'from_id': -22822305, 'owner_id': -22822305, 'date': 1640980531, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'copy_history': [{'id': 393, 'owner_id': 92933, 'from_id': 92933, 'date': 1640980164, 'post_type': 'post', 'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также автоматические переводы и автос

In [20]:
import requests  

def get_smm_index(gr,token):
    url = 'https://api.vk.com/method/groups.getMembers'  
    params = {  
        'group_id': gr,  
        'v': 5.95,  
        'access_token': token  
    }  
    response = requests.get(url, params = params)  
    membercount = response.json()['response']['count']  

    url = 'https://api.vk.com/method/wall.get'  
    params = {  
    'domain': gr,  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
    }  
    response = requests.get(url, params = params)  
    ds=response.json()['response']
    sum=0              
    for record in ds['items'][:10]:  
        sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    return sum/membercount

token = qq
get_smm_index('vk', token)

0.003020416259157319

In [21]:
sum=0     
count=0         
for record in ds['items'][:10]:
    sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    count+=1
    print(count,sum)  
pprint(sum/464)  

1 1740
2 3540
3 5747
4 10208
5 22651
6 24082
7 25458
8 26614
9 35491
10 36973
79.68318965517241


In [22]:
requests.get("http://www.google.com").status_code 

200